## Export to geojson

In [2]:
import glob
import json, geojson

In [3]:
filelist = glob.glob("./data/*coords.json")
print(filelist)

['./data/028295730_129_coords.json']


In [4]:
# Loads files
coords_file = filelist[0]
print(coords_file)
graph_file = coords_file.replace("_coords.json", "_chains.json")
thesis_file = coords_file.replace("_coords.json", "_thesis.json")
chaininfo_file = coords_file.replace("_coords.json", "_chaininfo.json")


with open(graph_file) as f:
    chains = json.load(f)

with open(chaininfo_file) as f:
    chaininfo = json.load(f)
    
print(len(chains), 'chains')

with open(coords_file) as f:
    coords = json.load(f)
    
with open(thesis_file) as f:
    thesis = json.load(f)
    
print(len(thesis), 'thesis')

./data/028295730_129_coords.json
13 chains
129 thesis


In [5]:
# Unit conversion
x_min, x_max = min(x[0] for x in coords.values()), max(x[0] for x in coords.values())
y_min, y_max = min(x[1] for x in coords.values()), max(x[1] for x in coords.values())

lat_zero, lon_zero = 10, 10
lat_range, lon_range = 40, 40

x_range = x_max - x_min
y_range = y_max - y_min

def to_angles(x, y):
    lat = (x - x_min)/x_range * lat_range + lat_zero
    lon = (y - y_min)/y_range * lon_range + lon_zero
    return lat, lon

In [10]:
chain_lines = []
for chain in chains:
    points = [geojson.Point(to_angles(*coords[node])) for node in chain]
    ls = geojson.LineString(points)
    feat = geojson.Feature(geometry=ls, properties=chaininfo[chain[0]] ) 
    chain_lines.append(feat)

line_collect = geojson.FeatureCollection(chain_lines)

dump = geojson.dumps(line_collect)

with open('./geojson/lines.geojson', 'w') as f:
    f.write(dump)

Feature Collections:

- lines: color, length
- thesis: nbr of thesis (i.e. director or not), searched?

Larger scale:
- city
- doctoral school

In [11]:
th_features = []
for th_id, info in thesis.items():
    point = geojson.Point(to_angles(*coords[th_id]))
    feat = geojson.Feature(geometry=point, properties=info)
    th_features.append(feat)
                           
th_features = geojson.FeatureCollection(th_features)

dump = geojson.dumps(th_features)

with open('./geojson/thesis.geojson', 'w') as f:
    f.write(dump)